## Implementation Details
- GuardrailAgent → Checks if the user query is AI-related.
- NewsFetcherAgent → Uses WebSearchTool to retrieve AI news.
- SummarizerAgent → Uses FunctionTool to summarize news.
- InsightsAgent → Uses FunctionTool to generate AI trends.
-  TriageAgent → Routes queries between the Summarizer and Insights Generator based on relevance.

## Import Libraries

In [1]:
import openai
from agents import( Agent, InputGuardrail, GuardrailFunctionOutput,
    InputGuardrailTripwireTriggered,
    RunContextWrapper,
    Runner,
    TResponseInputItem,
    input_guardrail, Runner, WebSearchTool, FunctionTool)

from pydantic import BaseModel, Field
import asyncio
from typing import List
import os

OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export


## Load API Key

In [2]:
openai_apikey = ""
os.environ["OPENAI_API_KEY"]=openai_apikey
openai.api_key = openai_apikey

## Step 1: Define the Guardrail Output Schema

In [3]:
class AIQueryOutput(BaseModel):
    is_ai_related: bool
    reasoning: str

## Step 2: Create the Guardrail Agent

In [4]:
guardrail_agent = Agent(
    name="Guardrail Check",
    instructions="Determine if the user query is related to AI advancements.",
    output_type=AIQueryOutput,
)

# Step 3: Define NewsFetcher Agent

In [5]:
web_search_tool = WebSearchTool()

news_fetcher_agent = Agent(
    name="NewsFetcher",
    handoff_description="Agent responsible for retrieving the latest AI news.",
    instructions="Use WebSearchTool to find the latest news on AI advancements.",
    tools=[web_search_tool],
)

## Step 4: Define Input Schema for Summarization

In [6]:
class Article(BaseModel):
    """Schema for a news article fetched by NewsFetcher."""
    title: str
    snippet: str
    url: str  # Optional: Add source URL for reference

class NewsResponse(BaseModel):
    """Response schema for the NewsFetcher agent."""
    articles: List[Article]

class SummaryResponse(BaseModel):
    """Response schema for the Summarizer agent."""
    summaries: List[str]

class InsightsResponse(BaseModel):
    """Response schema for the Insights agent."""
    insights: List[str]

## Step 5: Define Summarizer Function

In [7]:
def summarize_articles(news_response_json: str):
    print("\n [DEBUG] Summarizer received JSON input:\n", news_response_json)

    # Deserialize JSON into a Pydantic model
    try:
        news_response = NewsResponse.model_validate_json(news_response_json)
    except Exception as e:
        print("\n [ERROR] Summarizer: Invalid input format!\n", str(e))
        return SummaryResponse(summaries=["Error: Invalid input format."])

    print("\n[DEBUG] Summarizer parsed input correctly.")

    summaries = [
        f"Summary: {article.title} - {article.snippet}" for article in news_response.articles[:3]
    ]
    
    return SummaryResponse(summaries=summaries)

## Step 6: Create FunctionTool for Summarization

In [9]:
summarizer_tool = FunctionTool(
    name="Summarizer",
    description="Summarizes key points from AI-related news articles.",
    params_json_schema=SummaryResponse.model_json_schema(),
    on_invoke_tool=summarize_articles,
)

In [10]:
summarizer_agent = Agent(
    name="Summarizer",
    handoff_description="Agent responsible for summarizing news articles.",
    instructions="Summarize key points from news articles.",
    tools=[summarizer_tool],
)

## Step 8: Define Insights Generation Function

In [11]:
def generate_insights(summary_response_json: str):
    print(" [DEBUG] InsightsAgent received JSON input:\n", summary_response_json)

    # Deserialize JSON into a Pydantic model
    try:
        summary_response = SummaryResponse.model_validate_json(summary_response_json)
    except Exception as e:
        print(" [ERROR] InsightsAgent: Invalid input format!\n", str(e))
        return InsightsResponse(insights=["Error: Invalid input format."])

    print("\n[DEBUG] InsightsAgent parsed input correctly.")

    insights = [
        f"AI Trend Detected: {summary.split(':')[1].strip()}" for summary in summary_response.summaries if "AI" in summary
    ]
    
    return InsightsResponse(insights=insights)


## Step 9: Create FunctionTool for AI Insights

In [12]:
insights_tool = FunctionTool(
    name="InsightsGenerator",
    description="Generates AI trends and insights from summarized articles.",
    params_json_schema=InsightsResponse.model_json_schema(),
    on_invoke_tool=generate_insights,
)

## Step 10: Create an agent for AI Insights

In [13]:
insights_agent = Agent(
    name="InsightsAgent",
    handoff_description="Agent responsible for analyzing AI trends.",
    instructions="Generate insights and trends based on summarized AI news.",
    tools=[insights_tool],
)

## Step 11: Implement the Guardrail Function

In [14]:

@input_guardrail
async def ai_guardrail(ctx: RunContextWrapper[None], agent: Agent, input_data: str | list[TResponseInputItem]
) -> GuardrailFunctionOutput:
    result = await Runner.run(guardrail_agent, input_data, context=ctx.context)
    final_output = result.final_output_as(AIQueryOutput)
    
    # Debugging prints
    print(f"[Guardrail] User Input: {input_data}")
    print(f"[Guardrail] Validation Result: {final_output.is_ai_related} - {final_output.reasoning}")

    # Ensure correct logic
    return GuardrailFunctionOutput(
        output_info=final_output,
        tripwire_triggered=not final_output.is_ai_related,  # Only trigger if it's NOT AI-related
    )

## Step 12: Create the Triage Agent

In [15]:
triage_agent = Agent(
    name="TriageAgent",
    instructions="Route the query to either the Summarizer or Insights Agent based on AI relevance.",
    handoffs=[news_fetcher_agent, summarizer_agent, insights_agent],
    input_guardrails=[ ai_guardrail],
)


## Step 13: Execute the Hands-Off Workflow 

In [16]:
async def main(input):
    try:
        result = await Runner.run(triage_agent, input)
        print(result.final_output)
    except InputGuardrailTripwireTriggered:
        print("AI News Search guardrail tripped")  

## Test Case 1: Fetch AI News and Generate Insights

In [17]:
input1= "What are the latest advancements in AI?"
await main(input1)

[Guardrail] User Input: What are the latest advancements in AI?
[Guardrail] Validation Result: True - The query explicitly asks about the latest advancements in AI, which directly relates to developments in the field of artificial intelligence.
Artificial intelligence (AI) has experienced significant advancements recently, impacting various sectors and applications. Here are some of the latest developments:

**Enhanced Robotics Capabilities**

Companies like Google DeepMind and Microsoft have developed robots capable of complex tasks such as folding origami, organizing desks, and performing kitchen duties. These robots utilize large language models to adapt to diverse environments and learn rapidly from internet-sourced data. While challenges remain, such as improving autonomous human interactions and adapting to unpredictable settings, these advancements promise substantial benefits, including supporting a shrinking workforce and aiding in disaster recovery. ([ft.com](https://www.ft.c

## **Test Case 2: Unrelated Query (Should be Blocked)**

In [18]:

input2=  "How do I bake a cake?"
await main(input2)

[Guardrail] User Input: How do I bake a cake?
[Guardrail] Validation Result: False - The query is about baking a cake, which is a cooking task and does not involve artificial intelligence or its advancements.
AI News Search guardrail tripped
